# Unit conversions

## Available conversions

Time-of-flight neutron-scattering unit conversions implement the following conversions.
For background we refer to PaN-learning on [Introduction to neutron scattering](https://pan-learning.org/wiki/index.php/Introduction_to_neutron_scattering) and [Basics of neutron scattering](https://pan-learning.org/wiki/index.php/Basics_of_neutron_scattering).
In summary, in the table below for the supported unit conversions:

- $h$ is the Planck constant, $m_{\mathrm{n}}$ is the neutron mass.
- $L_1$ is the primary flight path length, i.e., the distance between source and sample.
  The source position defines the point where $t=0$ (or vice versa).
- $L_2$ is the secondary flight path length, i.e., the distance(s) between sample and detector(s).
- $L_{\mathrm{total}} = L_1 + L_2$
- $d$ is the interplanar lattice spacing
- $\theta$ is the scattering angle, as defined in Bragg's law, not to be confused with $\theta_{\mathrm{spherical}}$ in sperical coordinates.
  In many instruments the beam is roughly aligned with the $z$-axis, so $2\theta = \theta_{\mathrm{spherical}}$.
  The factor of $2$ is a recurrent source of bugs.
  
In the special case of inelastic neutron scattering we have additionally:

- $E_i$ is the known incident energy. This case is called *direct inelastic scattering*.
  We define $t_0 = \sqrt{L_1^2 \cdot m_{\mathrm{n}} / E_i}$
- $E_f$ is the known final energy. This case is called *indirect inelastic scattering*.
  We define $t_0 = \sqrt{L_2^2 \cdot m_{\mathrm{n}} /E_F}$
  In this case $E_f$ is actually determined by analyzer crystals in the secondary flight path.
  It is assumed that the detector positions are set to the effective (neutronic) rather than physical positions, since the computation of $L_2$ assumes a straight line connection between sample and detectors.

|Origin unit |Target unit |Formula  used for coord conversion |
|---|---|:---|
|`tof`|`dspacing`|$d = \frac{\lambda}{2\sin(\theta)} = \frac{h \cdot t}{L_{\mathrm{total}} \cdot m_{\mathrm{n}} \cdot 2 \sin(\theta)}$|
|`dspacing`|`tof`|$t = \frac{d \cdot L_{\mathrm{total}} \cdot m_{\mathrm{n}} \cdot 2 \sin(\theta)}{h}$|
| `tof`|`wavelength`|$\lambda = \frac{h \cdot t}{m_{\mathrm{n}} \cdot L_{\mathrm{total}}}$|
|`wavelength`|`tof`|$t = \frac{m_{\mathrm{n}} \cdot L_{\mathrm{total}} \cdot \lambda}{h}$|
| `tof`|`energy`|$E = \frac{m_{\mathrm{n}}L^2_{\mathrm{total}}}{2t^2}$|
| `tof`|`energy_transfer` (direct)|$E = E_i - \frac{m_{\mathrm{n}}L^2_{\mathrm{2}}}{2\left(t-t_0\right)^{2}}$|
| `tof`|`energy_transfer` (indirect)|$E = \frac{m_{\mathrm{n}}L^2_{\mathrm{1}}}{2\left(t-t_0\right)^{2}} - E_f$|
|`energy`|`tof`|$t = \frac{L_{\mathrm{total}}}{\sqrt{\frac{2 E}{m_{\mathrm{n}}}}}$|

## Beamline geometry parameters used in unit conversions

In [ ]:
from graphviz import Digraph
dot = Digraph()
dot.edge('source_position', 'incident_beam')
dot.edge('sample_position', 'incident_beam')
dot.edge('position', 'scattered_beam')
dot.edge('sample_position', 'scattered_beam')
dot.edge('incident_beam', 'L1', label='norm')
dot.edge('scattered_beam', 'L2', label='norm')
dot.edge('L1', 'Ltotal')
dot.edge('L2', 'Ltotal')
dot.edge('incident_beam', 'two_theta')
dot.edge('scattered_beam', 'two_theta')
dot.node('wavelength', shape='box', style='filled', color='lightgrey')
dot.node('energy', shape='box', style='filled', color='lightgrey')
dot.node('energy_transfer (direct)', shape='box', style='filled', color='lightgrey')
dot.node('energy_transfer (indirect)', shape='box', style='filled', color='lightgrey')
dot.node('dspacing', shape='box', style='filled', color='lightgrey')
dot.node('Q', shape='box', style='filled', color='lightgrey')
dot.edge('Ltotal', 'energy')
dot.edge('L1', 'energy_transfer (direct)')
dot.edge('L2', 'energy_transfer (direct)')
dot.edge('incident_energy', 'energy_transfer (direct)')
dot.edge('L1', 'energy_transfer (indirect)')
dot.edge('L2', 'energy_transfer (indirect)')
dot.edge('final_energy', 'energy_transfer (indirect)')
dot.edge('Ltotal', 'wavelength')
dot.edge('Ltotal', 'dspacing')
dot.edge('two_theta', 'dspacing')
dot.edge('two_theta', 'Q')
dot.edge('wavelength', 'Q')
scattering = dot

dot = Digraph()
dot.edge('source_position', 'Ltotal')
dot.edge('position', 'Ltotal')
dot.node('wavelength', shape='box', style='filled', color='lightgrey')
dot.node('energy', shape='box', style='filled', color='lightgrey')
dot.edge('Ltotal', 'energy')
dot.edge('Ltotal', 'wavelength')
nonscattering = dot

def show_geometry_params(scatter):
    if scatter:
        display(scattering)
    else:
        display(nonscattering)

The parameters given in the table above are retrieved from the coordinates or attributes of the input data.
The following graph illustrates which coords (or attrs) are known and used by `convert`.
Read the graph as follows:

- Ellipses represent geometry-related parameters of the beamline, e.g., `two_theta`.
- The grey rectangles denote an input or output unit, e.g., `wavelength`.
- A unit conversion involving a particular input or output unit will require the parameters connected to the rectangle by arrows.
- If a unit conversion requires a parameter the tree of paramameter ellipses is searched bottom-up.

Example:

- Consider a unit conversion to `wavelength` (from, e.g., `tof`) *or* from `wavelength` (to, e.g., `tof`) will require the parameters connected to the rectangle by arrows, in this case `Ltotal`.
- If a coord or attr with name `Ltotal` is found it is used for the conversion.
- If `Ltotal` is not found it will be computed based on coords or attrs `L1` and `L2`.
- If, e.g., `L2` is not found it is computed as the norm of the `scattered_beam` coord or attr.
- If, `scattered_beam` is not found it us computed as `position - sample_position`.

There are two cases, scattering conversion and non-scattering conversion (e.g., for beam monitors or for imaging beamlines).

Case 1: `convert(..., scatter=True)`

In [ ]:
show_geometry_params(scatter=True) # helper defined in hidden cell

Case 2: `convert(..., scatter=False)`

In [ ]:
show_geometry_params(scatter=False) # helper defined in hidden cell